In [1]:
import sys 
import os


sys.path.append(os.path.abspath(".."))

In [2]:
from src.data.load_data import load_raw_data

df=load_raw_data()

df.head()

,jira_created_at,work_start_at,pr_created_at,merged_at,pr_id,author_id,author_seniority,service_name,jira_issue_key,jira_type,...,is_legacy_codebase,coding_time_hours,review_time_hours,rework_time_hours,cycle_time_total,change_failure,trace_id,error_log,root_cause_category,failed_pipeline_step
0,2024-01-01 00:05:00,2024-01-02 18:19:32,2024-01-02 19:13:07,2024-01-02 19:46:10,PR_10000,DEV_040,Junior,Email-Dispatcher,APM-1000,Story,...,0,0.89,0.35,0.0,1.44,0,NaN,NaN,NaN,NaN
1,2024-01-01 00:15:00,2024-01-02 06:05:14,2024-01-02 07:15:18,2024-01-02 11:12:37,PR_10001,DEV_000,Mid,Notification-Broker,APM-1001,Tech Debt,...,1,1.17,2.00,0.0,5.12,0,NaN,NaN,NaN,NaN
2,2024-01-01 00:37:00,2024-01-02 02:35:50,2024-01-02 06:32:15,2024-01-02 15:00:10,PR_10002,DEV_044,Junior,Auth-Service,APM-1002,Tech Debt,...,1,3.94,4.96,0.0,12.41,1,TRC-ce59ec948b70,AssertionError: Auth-Service received unexpect...,AI_Hallucination_Logic,Unit_Test_Execution
3,2024-01-01 00:56:00,2024-01-02 06:17:52,2024-01-02 09:45:30,2024-01-03 01:20:44,PR_10003,DEV_020,Mid,Recommendation-Engine,APM-1003,Task,...,0,3.46,3.35,0.0,19.05,1,TRC-66fd4ac1eb13,AssertionError: Recommendation-Engine received...,AI_Hallucination_Logic,Unit_Test_Execution
4,2024-01-01 01:04:00,2024-01-02 07:23:44,2024-01-02 10:27:49,2024-01-02 19:33:32,PR_10004,DEV_033,Senior,Search-Optimizer,APM-1004,Story,...,0,3.07,2.15,0.0,12.16,0,NaN,NaN,NaN,NaN
